In [1]:
import cv2
import os
import numpy as np

In [2]:
rawdata = '../../datasets/rawdata/'
dataset = '../../datasets/to_train/preprocessed/'

In [17]:
def face_detect(img, path):
    # Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(200, 200))

    if len(faces) > 0:
      x, y, w, h = faces[0]  # Menggunakan wajah pertama yang terdeteksi
      # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
      face = img[y:y+h, x:x+w]  # Memotong bagian wajah

      if face.size > 0:
        face = cv2.resize(face, (224, 224))
        cv2.imwrite(path, face)
        return True
      else:
        print("Area Wajah Tidak Terdeteksi " + path)
        return False
    else:
      print("Wajah Tidak Terdeteksi " + path)
      return False

In [18]:
fail_detect = 0

for npm_folder in os.listdir(rawdata):
    npm_path = os.path.join(rawdata, npm_folder)

    if os.path.isdir(npm_path):
        # Membuat folder NPM di dalam folder "dataset"
        npm_dataset_folder = os.path.join(dataset, npm_folder)
        os.makedirs(npm_dataset_folder, exist_ok=True)

        # Loop melalui semua file gambar di folder NPM
        for img_file in os.listdir(npm_path):
            img_path = os.path.join(npm_path, img_file)

            if os.path.isfile(img_path):
                # Deteksi wajah dan simpan ke folder NPM di "dataset"
                output_path = os.path.join(npm_dataset_folder, img_file)
                img = cv2.imread(img_path)
                result = face_detect(img, output_path)
                if not result:
                    fail_detect += 1
            else:
                print("Failed to load image: " + img_path)
                fail_detect += 1

print('Jumlah foto yang tidak terdeteksi:', fail_detect)

Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2007051040\IMG-20231116-WA0022.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051001\WIN_20230930_10_53_51_Pro.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20190615-WA0010.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20220609-WA0018.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20231211-WA0012.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20231211-WA0013.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20231211-WA0015.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051006\IMG-20231211-WA0017.jpg
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051017\20230713_103229.png
Wajah Tidak Terdeteksi ../../datasets/to_train/preprocessed/2017051017\IMG-20230302-WA0016.png
Wajah Tidak Terdeteksi ../../datasets/to_train/p